In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [24]:
# load text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='latin-1').read()
#raw_text = open(filename).read()
raw_text = raw_text.lower() #vocabulary reduction

In [25]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [26]:
print(chars)
print(char_to_int)
print(int_to_char)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x91', '\x92', '\x93', '\x94']
{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, '0': 9, '3': 10, ':': 11, ';': 12, '?': 13, '[': 14, ']': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42, '\x91': 43, '\x92': 44, '\x93': 45, '\x94': 46}
{0: '\n', 1: ' ', 2: '!', 3: '(', 4: ')', 5: '*', 6: ',', 7: '-', 8: '.', 9: '0', 10: '3', 11: ':', 12: ';', 13: '?', 14: '[', 15: ']', 16: '_', 17: 'a', 18: 'b', 19: 'c', 20: 'd', 21: 'e', 22: 'f', 23: 'g', 24: 'h', 25: 'i', 26: 'j', 27: 'k', 28: 'l', 29: 'm', 30: 'n', 31: 'o', 32: 'p', 33: 'q', 34: 'r', 35: 's', 36: 't',

In [27]:
import pandas as pd
raw_text[:10]

'*** start '

In [28]:
len(chars)

47

In [29]:
len(raw_text)

144515

In [30]:
seq_in = raw_text[1:1 + 50]
print("seq_in:",seq_in)
seq_out = raw_text[51]
print("seq_out:",seq_out)

seq_in: ** start of this project gutenberg ebook alices a
seq_out: d


In [31]:
dataX=[]
print(seq_in)
dataX.append([char_to_int[char] for char in seq_in])
print(dataX)

** start of this project gutenberg ebook alices a
[[5, 5, 1, 35, 36, 17, 34, 36, 1, 31, 22, 1, 36, 24, 25, 35, 1, 32, 34, 31, 26, 21, 19, 36, 1, 23, 37, 36, 21, 30, 18, 21, 34, 23, 1, 21, 18, 31, 31, 27, 1, 17, 28, 25, 19, 21, 44, 35, 1, 17]]


In [32]:
dataY = []
print(seq_out)
dataY.append(char_to_int[seq_out])
print(dataY)

d
[20]


In [33]:
seq_length = 100
dataX, dataY = [], []

In [34]:
for i in range(0, len(raw_text) - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [35]:
n_patterns = len(dataX)
print(n_patterns)

144415


In [36]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
#X

In [37]:
# normalize
X = X / float(len(chars))
#print(X)

In [38]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, categories='auto')
y = encoder.fit_transform(np.array(dataY).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [40]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(256))
model.add(Dropout(0.2))

model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [41]:
# Train the model (you can adjust epochs and batch_size)
model.fit(X, y, epochs=10, batch_size=2000, verbose=1)

Epoch 1/10
73/73 [==============================] - 1004s 14s/step - loss: 3.1323
Epoch 2/10
73/73 [==============================] - 995s 14s/step - loss: 3.0569
Epoch 3/10
73/73 [==============================] - 996s 14s/step - loss: 3.0314
Epoch 4/10
73/73 [==============================] - 997s 14s/step - loss: 2.9563
Epoch 5/10
73/73 [==============================] - 1009s 14s/step - loss: 2.8644
Epoch 6/10
73/73 [==============================] - 997s 14s/step - loss: 2.7761
Epoch 7/10
73/73 [==============================] - 999s 14s/step - loss: 2.7150
Epoch 8/10
73/73 [==============================] - 1006s 14s/step - loss: 2.6568
Epoch 9/10
73/73 [==============================] - 1003s 14s/step - loss: 2.6055
Epoch 10/10
73/73 [==============================] - 999s 14s/step - loss: 2.5590


In [67]:
#generating the text
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


Seed:
" only knew how to begin.
for, you see, so many out-of-the-way things had happened lately,
that alice "


In [77]:
#generate based on the input text
def generate_text(seed_text, n_chars=100):
    seed_text=seed_text.lower()
    generated_text = seed_text

    for i in range(n_chars):
        x_pred = np.array([char_to_int[c] for c in seed_text])
        x_pred = np.reshape(x_pred, (1, len(x_pred), 1)) / float(len(chars))

        prediction = model.predict(x_pred, verbose=0)
        index = np.argmax(prediction)
        char_out = int_to_char[index]

        generated_text += char_out
        seed_text = seed_text[1:] + char_out


    return generated_text



In [78]:
print(generate_text("Alice was beginning",200)) # I don't know how to fix this problem, and I found from the comments there are a lot of people hving same question

alice was beginning to the cane the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the taid the tai
